In [2]:
import cv2
import numpy
import os
import keyboard
import csv

#get workingdirectory
workdir=input('Path of video file to analyze:')
file=input('Name of video file to analyze:')
videofile=os.path.join(workdir,file)
resultfolder=input('Name of result folder')
#record behavior strattime and stoptime,unit:s#
sniff=[]
chase=[]
mount=[]
intromission=[]
ejaculation=[]
##############
font=cv2.FONT_HERSHEY_SCRIPT_COMPLEX #设置视频中的字体
video = cv2.VideoCapture(videofile)
text=['none','sniff','chase','mount','intromission','ejaculation']#需要打标的几个动作#
# 获得帧率及尺寸
fps = video.get(cv2.CAP_PROP_FPS)
size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fNUMS = video.get(cv2.CAP_PROP_FRAME_COUNT)
seccount = int(fNUMS/fps)  # 计算出视频总时长，单位：秒
def onTrackbarSlide(pos):
    global video
    video.set(0, pos * 1000)
# 读帧
c=0 #第几帧的index
b=0 #behavior的index
behav=text[0]
cv2.namedWindow('mywindow', cv2.WINDOW_NORMAL)  # 创建窗口
cv2.createTrackbar('TimePos', 'mywindow', 0, seccount, onTrackbarSlide)  # 创建滑动条和回调函数
video.set(cv2.CAP_PROP_POS_FRAMES,c)
success, frame = video.read()
while success:
    frame=cv2.putText(frame,behav,(10,50),font,3,(0,255,255),2,cv2.LINE_AA)
    curtime=int(c/fps)
    cv2.imshow('mywindow', frame)  # 显示
    cv2.setTrackbarPos('TimePos', 'mywindow', curtime)  # 更新滑动条上的位置
    #读取键盘值
    key = cv2.waitKey(1000000000)
    if keyboard.is_pressed('.'):
        c=c+3
        video.set(cv2.CAP_PROP_POS_FRAMES,c)
        success, frame = video.read()
    if keyboard.is_pressed(','):
        c=c-3
        video.set(cv2.CAP_PROP_POS_FRAMES,c)
        success, frame = video.read()
    if keyboard.is_pressed('s'): #sniff hotkey:s
        sniff.append(c/fps)
        if len(sniff)%2==1:
            behav=text[1]
        if len(sniff)%2==0:
            behav=text[0]
    if keyboard.is_pressed('c'): #sniff hotkey:c
        chase.append(c/fps)
        if len(chase)%2==1:
            behav=text[2]
        if len(chase)%2==0:
            behav=text[0]
    if keyboard.is_pressed('m'): #sniff hotkey:s
        mount.append(c/fps)
        if len(mount)%2==1:
            behav=text[3]
        if len(mount)%2==0:
            behav=text[0]
    if keyboard.is_pressed('i'): #sniff hotkey:s
        intromission.append(c/fps)
        if len(intromission)%2==1:
            behav=text[4]
        if len(intromission)%2==0:
            behav=text[0]
    if keyboard.is_pressed('e'): #sniff hotkey:s
        ejaculation.append(c/fps)
        if len(ejaculation)%2==1:
            behav=text[4]
        if len(ejaculation)%2==0:
            behav=text[0]
    #设置Q按下时退出
    if key == ord("q"):
        break
    # cv2.waitKey(int(1000 / int(fps)))  # 设置每一帧imshow的持续时间
    #一帧需要多长时间，即1000ms除以帧率
    
video.release()
cv2.destroyAllWindows()
###calculate behavior duration###
ifout=1 #用于判断打标的数据是否可以被输出
##firstly,check whether strattime and stoptime are paired
if len(sniff)%2==1:
    print('sniff starttime and stoptime are not paired')
    ifout=0
if len(chase)%2==1:
    print('chase starttime and stoptime are not paired')
    ifout=0
if len(intromission)%2==1:
    print('intromission starttime and stoptime are not paired')
    ifout=0
if len(mount)%2==1:
    print('mount starttime and stoptime are not paired')
    ifout=0
if len(ejaculation)%2==1:
    print('ejaculation starttime and stoptime are not paired')
    ifout=0
##calculate behavior duration
sniff_dur=0
chase_dur=0
mount_dur=0
intromission_dur=0
ejaculation_dur=0
if ifout==1:
    sniff_fre=len(sniff)/2
    chase_fre=len(chase)/2
    mount_fre=len(mount)/2
    intromission_fre=len(intromission)/2
    ejaculation_fre=len(ejaculation)/2
    for i in range(0,len(sniff),2):
        sniff_dur=sniff_dur+(sniff[i+1]-sniff[i])
    for i in range(0,len(chase),2):
        chase_dur=chase_dur+(chase[i+1]-chase[i])
    for i in range(0,len(mount),2):
        mount_dur=mount_dur+(mount[i+1]-mount[i])
    for i in range(0,len(intromission),2):
        intromission_dur=intromission_dur+(intromission[i+1]-intromission[i])
    for i in range(0,len(ejaculation),2):
        ejaculation_dur=ejaculation_dur+(ejaculation[i+1]-ejaculation[i])
##Output tagging results
    datafolder=os.path.join(workdir, resultfolder)
    os.makedirs(datafolder)
    colnames1=['sniff_duration','sniff_frequency','chase_duration','chase_frequency','mount_duration','mount_frequency','intromission_duration','intromission_frequency','ejaculation_duration','ejaculation_frequency']
    with open(datafolder + '/fre_dur.csv', 'w', newline='') as results:
        writer = csv.writer(results)
        writer.writerow(colnames1)
        writer.writerow([sniff_dur,sniff_fre,chase_dur,chase_fre,mount_dur,mount_fre,intromission_dur,intromission_fre,ejaculation_dur,ejaculation_fre])
    colnames2=['timepoit','behavior']
    with open(datafolder + '/timepoint.csv', 'w', newline='') as results:
        writer = csv.writer(results)
        writer.writerow(colnames2)
        for i in range(0,len(sniff)):
            writer.writerow([sniff[i],'sniff'])
        for i in range(0,len(mount)):
            writer.writerow([mount[i],'mount'])
        for i in range(0,len(chase)):
            writer.writerow([chase[i],'chase'])
        for i in range(0,len(intromission)):
            writer.writerow([intromission[i],'intromission'])
        for i in range(0,len(ejaculation)):
            writer.writerow([ejaculation[i],'ejaculation'])